In [ ]:

import boto3
import numpy as np
from astropy.io import fits
import gzip
import re
from io import BytesIO


def list_folders_in_bucket(bucket_name):
    s3 = boto3.client('s3')
    
    folders = set()
    paginator = s3.get_paginator('list_objects_v2')
    for result in paginator.paginate(Bucket=bucket_name, Delimiter='/'):
        # Collect common prefixes (which represent "folders")
        if 'CommonPrefixes' in result:
            for prefix in result['CommonPrefixes']:
                folder_name = prefix['Prefix'].split('/')[0]
                # Check if the folder matches the pattern 'e' followed by a five-digit integer
                if re.match(r'^e\d{5}$', folder_name):
                    folders.add(folder_name)
    
    return list(folders)

bucket_name = 'backplanetest'
folder_list = list_folders_in_bucket(bucket_name)
print(len(folder_list))

In [ ]:
sorted_folders = sorted(folder_list)

In [ ]:
subset = sorted_folders[0:6]

In [ ]:
# for the folders, make an image from first 100 frames 
import numpy as np
from astropy.io import fits
import gzip


s3 = boto3.client('s3')

def download_fits(bucket_name, key):
    response = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.open(BytesIO(response['Body'].read()), 'rb') as f:
        return fits.open(BytesIO(f.read()))

bucket_name = 'backplanetest'

num_images = 5

image_list = []

# file name format 
# s3://backplanetest/e00759/e00759-nd-b00-f0001-t00000-g_dose.fits.gz

for i in subset:
    print(key)
    key = f"{i}/{i}-nd-b00-f0001-t00000-g_dose.fits.gz"  # Assuming files are named like 'fits_image_1.fits', 'fits_image_2.fits', etc.
    image = download_fits(bucket_name, key)
    image_list.append(image[0].data)

# Stack the images
stacked_image = np.stack(image_list, axis=0)

# Save the stacked image as a FITS file
fits.writeto('stacked_image.fits', stacked_image, overwrite=True)

print("Stacked image saved as 'stacked_image.fits'")

In [ ]:
# Initialize S3 client
s3 = boto3.client('s3')

subset = sorted_folders[1000:2001]
# Function to download FITS image from S3
def download_fits(bucket_name, key):
    try:
        response = s3.get_object(Bucket=bucket_name, Key=key)
        with gzip.open(BytesIO(response['Body'].read()), 'rb') as f:
            return fits.open(BytesIO(f.read()))    
    except Exception as e:
        if 'NoSuchKey' in str(e):
            print(f"Key '{key}' does not exist. Skipping...")
            return None
        else:
            print("something else went wrong")
            print(str(e))
            return None
# Bucket name
bucket_name = 'backplanetest'

# Initialize an array to hold the sum of pixel values
sum_image = None

# Add the pixel values of each image
for i in subset:
    key = f"{i}/{i}-nd-b00-f0001-t00000-g_dose.fits.gz"  # Assuming files are named like 'fits_image_1.fits', 'fits_image_2.fits', etc.
    image = download_fits(bucket_name, key)
    print(key)
    if image is not None:
        if sum_image is None:
            sum_image = np.zeros_like(image[0].data, dtype=np.float64)
        sum_image += image[0].data.astype(np.float64)

# Save the summed image as a FITS file
fits.writeto('summed_1000_image.fits', sum_image, overwrite=True)

print("Summed image saved as 'summed_image_2.fits'")

In [ ]:
# Initialize S3 client
s3 = boto3.client('s3')

subset = sorted_folders[1000:2001]
# Function to download FITS image from S3
def download_fits(bucket_name, key):
    try:
        response = s3.get_object(Bucket=bucket_name, Key=key)
        with gzip.open(BytesIO(response['Body'].read()), 'rb') as f:
            return fits.open(BytesIO(f.read()))    
    except Exception as e:
        if 'NoSuchKey' in str(e):
            print(f"Key '{key}' does not exist. Skipping...")
            return None
        else:
            print("something else went wrong")
            print(str(e))
            return None
# Bucket name
bucket_name = 'backplanetest'

for group in groups_of_100: 
    # Initialize an array to hold the sum of pixel values
    sum_image = None

    # Add the pixel values of each image
    for i in group:
        key = f"{i}/{i}-nd-b00-f0001-t00000-g_dose.fits.gz"  # Assuming files are named like 'fits_image_1.fits', 'fits_image_2.fits', etc.
        image = download_fits(bucket_name, key)
        print(key)
        if image is not None:
            if sum_image is None:
                sum_image = np.zeros_like(image[0].data, dtype=np.float64)
            sum_image += image[0].data.astype(np.float64)

    # Save the summed image as a FITS file
    fits.writeto(f'{i}_g100_image.fits', sum_image, overwrite=True)

    print(f"Summed image saved as '{i}_g100_image.fits'")

In [ ]:
subset_list = sorted_folders[:1000]

groups_of_100 = [subset_list[i:i+100] for i in range(0, len(subset_list), 100)]


groups_of_100

In [ ]:
len(groups_of_100)